Ide: tag dansk-sproget wikipedia, find alle brugere og lav netværk over brugere, der har redigeret de samme artikler. Derefter, få lix-tal eller lignende fra alle artikler, og se om der er sammenhæng... Eller groupér efter konnektivitet, og se på sammenhængen i grupperne baseret på keywords i artiklerne, eller fordelingen af kategorier.

X Filter navne med bot eller ip-adresser.
X Få data
X Konstruer graf
X Cluster
O Test clusters for modularity mod random graphs
O Test text similarity mellem og indenfor clusters
O Find kategorier for clusters

In [32]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
from itertools import combinations
from functools import reduce
from collections import defaultdict
import re
import numpy as np
from sklearn.cluster import SpectralClustering
from sklearn import metrics
import pywikibot
import networkx as nx
from tqdm import tqdm
from joblib import Parallel, delayed
from pywikibot import pagegenerators
import nltk
from nltk.tokenize import MWETokenizer
from nltk.corpus import stopwords
from nltk import word_tokenize
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [145]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nikol\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nikol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

### Getting data

In [44]:
active_users_url = 'https://da.wikipedia.org/w/index.php?title=Speciel:Aktive_Brugere&\
                    username=&wpFormIdentifier=specialactiveusers&limit=1000'
r = requests.get(active_users_url)
soup = BeautifulSoup(r.content)

In [45]:
user_elements = soup.find_all('a', {'class':['mw-userlink', 'new mw-userlink']})
users = [element.text for element in user_elements]
users = users[:20]
len(users)

20

In [2]:
def isipadress(string):
    ip_regex = r"\b((25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)(\.|$)){4}\b"
    if len(string) < 5:
        return False
    if re.findall(ip_regex, string) or string[4] == ':':
        return True
    return False
    
def isbot(string):
    return 'bot' in string.lower()

def get_user_contributions(username):
    site = pywikibot.Site('da', 'wikipedia')
    user = pywikibot.User(site, username)
    contributions = user.contributions(total=500)
    articles = [contrib[0].title() for contrib in contributions 
                if ':' not in contrib[0].title()]
    return articles

def get_page_contributors(pagename):
    site = pywikibot.Site('da', 'wikipedia')
    page = pywikibot.Page(site, pagename)
    contributors = page.contributors()
    # Implement botfilter
    usernames = [user for user in contributors 
                 if not (isbot(user) or isipadress(user))]
    return {pagename: usernames}

In [15]:
site = pywikibot.Site('da', 'wikipedia')
allpagesgen = site.allpages()
all_bots = []
for i in tqdm(range(100)):
    page = next(iter(allpagesgen))
    contributors = list(page.contributors())
    bots = [c for c in contributors if isbot(c)]
    all_bots += bots
len(all_bots)

100%|██████████| 100/100 [00:20<00:00,  4.84it/s]


366

In [29]:
site = pywikibot.Site('da', 'wikipedia')
allpagesgen = site.allpages()
def get_bots(page):
    contributors = list(page.contributors())
    bots = [c for c in contributors if isbot(c)]
    return bots

parallel = Parallel(n_jobs=-2, return_as = "generator")
all_bots = parallel(delayed(get_bots)(page)
                               for i, page in enumerate(allpagesgen))
all_bots_list = []
counter = 0
print_every = 1000
break_when = 10000
while True:
    try:
        all_bots_list += next(all_bots)
        counter += 1
        if counter % print_every == 0:
            print(counter)
        if counter == break_when:
            break
    except StopIteration:
        print('Done')
        break
        
print('All bots found: ', len(all_bots_list))
print('Unique bots: ', len(set(all_bots_list)))

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
All bots found:  137309
Unique bots:  201


In [33]:
unique_bots = list(set(all_bots_list))
bot_data = pd.DataFrame(unique_bots, columns = ['name'])
def get_bot_data(bot_name):
    bot = pywikibot.

,name
0,PixelBot
1,Broadbot
2,Chlewbot
3,EivindBot
4,Ptbotgourou
...,...
196,YiFeiBot
197,Manubot
198,Steenthbot
199,GnawnBot


In [37]:
list(pywikibot.User(site, 'PixelBot').contributions())

[(Page('QRpedia'),
  6872737,
  Timestamp(2013, 3, 7, 6, 1, 54),
  'r2.7.3) (Robot tilføjer [[hy:Քյուերպեդիա]]'),
 (Page('Q'),
  6872735,
  Timestamp(2013, 3, 7, 5, 51, 58),
  'r2.7.3) (Robot tilføjer ace, af, als, an, ar, arc, ast, az, bat-smg, be, be-x-old, bg, br, bs, ca, co, cs, cy, de, diq, el, en, eo, es, et, eu, fa, fi, fr, fur, fy, gan, gd, gl, gv, he, hr, ht, hu, id, ilo, is, it, ja, ka, ko, ku, kw, la, lt, lv, mhr,...'),
 (Page('PROM'),
  6872188,
  Timestamp(2013, 3, 6, 20, 49, 50),
  "r2.7.3) (Robot: Ændrer [[uk:Програмовний постійний запам'ятовувач]] til [[uk:PROM]]"),
 (Page('Preservation and Long-term Access through Networked Services'),
  6871812,
  Timestamp(2013, 3, 6, 17, 30, 22),
  'r2.7.3) (Robot: Ændrer [[en:Planets Project]] til [[en:Framework Programmes for Research and Technological Development]]'),
 (Page('PowerPC'),
  6871459,
  Timestamp(2013, 3, 6, 14, 24, 28),
  'r2.7.3) (Robot: Ændrer [[fa:پاور پی\u200cسی]] til [[fa:پاور پی\u200cسی (ریز پردازنده)]]'),
 (P

In [ ]:
print('Getting user articles')
user_articles = Parallel(n_jobs=-2)(delayed(get_user_contributions)(user)
                                    for user in tqdm(users))

all_articles = set()
for articles in user_articles:
    all_articles = all_articles | set(articles)
all_articles = list(all_articles)

print('Getting all contributors')
page_contribs = Parallel(n_jobs=-2)(delayed(get_page_contributors)(article)
                                    for article in tqdm(all_articles))

In [243]:
len(all_articles)

426

In [219]:
page_contribs = reduce(lambda a, b: {**a, **b}, page_contribs)
print(page_contribs)

{'Olaf Eller': ['Olaf Eller', 'Pixi Uno', 'Sarrus', 'AEJ'], 'Potentiel energi': ['Glenn', 'Inc', 'AEJ', 'Hjart', 'Biscuit-in-Chief', 'MacApps', 'Rodejong', 'KnudW', 'Crudiant', 'Savfisk', 'Pugilist', 'Strawa', 'Fodbold kongen', 'Christian Giersing', 'Palnatoke', 'Nielssonnich', 'Pred', 'HenrikMidtiby', 'GunnerPoulsen', 'Hashar', 'Ilario', 'Sten', 'Peo'], 'Josephine Park': ['Hjart', 'Dipsacus fullonum', 'AndersViborg', 'Amjaabc', 'Laanders', 'Siksebuffen', 'Anne-Sophie Ofrim', 'Tjernobyl', 'TherasTaneel', 'AEJ', 'Toxophilus', 'Tjalland', 'CeciTGreg', 'Pugilist'], 'Superisligaen': ['Henrik Hansen', 'Hjart', 'PHE77', 'AEJ', 'Mikkelh96', 'Tøndemageren', 'SorenRK', 'CarlKlogeOve', 'Rodejong', 'Izm0', 'Patchfinder', 'Royal Export', 'Pandikas', 'STUDENT57', 'Veolia', 'Kaare', 'Palnatoke', 'EPO', 'Sarrus', 'Altaïr', 'Trade', 'Steenth', 'Amjaabc', 'Christian75', 'Pugilist', 'Hockeyindustrien', 'Ejwin', 'Ultraman', 'Viby27', 'Ohkami', 'C.Thure', 'Barklund'], 'VM i ishockey 1962': ['AEJ', 'Henrik

### Graph construction

In [220]:
edge_dict = defaultdict(lambda: {'pages':[],'n_pages':0})

for page, contribs in page_contribs.items():
    if len(contribs) < 2:
        continue
    contribs.sort()
    contrib_pairs = list(combinations(contribs, r=2))
    for contrib_pair in contrib_pairs:
        edge_dict[contrib_pair]['pages'].append(page)
        edge_dict[contrib_pair]['n_pages'] += 1

G = nx.Graph()
G.add_edges_from(edge_dict.keys())
nx.set_edge_attributes(G, edge_dict)

In [93]:
nx.get_edge_attributes(G, 'pages')

{('AEJ', 'Olaf Eller'): ['Olaf Eller'],
 ('AEJ', 'Pixi Uno'): ['Olaf Eller', 'Kasper Lorentzen'],
 ('AEJ', 'Sarrus'): ['Olaf Eller',
  'Superisligaen',
  'VM i ishockey 1962',
  'Svenstrup (Aalborg Kommune)',
  'Kasper Lorentzen',
  'Roy Keane'],
 ('AEJ', 'Biscuit-in-Chief'): ['Potentiel energi', 'Saudi-Arabien'],
 ('AEJ', 'Christian Giersing'): ['Potentiel energi',
  'Svenstrup (Aalborg Kommune)',
  'Oskarshamn',
  'Forsmark atomkraftværk'],
 ('AEJ', 'Crudiant'): ['Potentiel energi'],
 ('AEJ', 'Fodbold kongen'): ['Potentiel energi'],
 ('AEJ', 'Glenn'): ['Potentiel energi',
  'Jean Hersholt',
  'Saudi-Arabien',
  'Forsmark atomkraftværk'],
 ('AEJ', 'GunnerPoulsen'): ['Potentiel energi'],
 ('AEJ', 'Hashar'): ['Potentiel energi', 'Saudi-Arabien'],
 ('AEJ', 'HenrikMidtiby'): ['Potentiel energi'],
 ('AEJ', 'Hjart'): ['Potentiel energi',
  'Josephine Park',
  'Superisligaen',
  'Henrik Dalsgaard',
  'Saudi-Arabien'],
 ('AEJ', 'Ilario'): ['Potentiel energi'],
 ('AEJ', 'Inc'): ['Potentiel ene

In [99]:
page_contribs['Saudi-Arabien']

['AEJ',
 'Akkuratesse',
 'Allano',
 'Amjaabc',
 'Apw',
 'Artikler',
 'BBC',
 'Biscuit-in-Chief',
 'BjornGraabek',
 'BrianHansen',
 'Brotta8',
 'Byrial',
 'Cgt',
 'CommonsDelinker',
 'Dipsacus fullonum',
 'EPO',
 'Gereon K.',
 'Glenn',
 'Haabet',
 'HansJensen',
 'Harne',
 'Hashar',
 'Hede2000',
 'Hejsa',
 'Hjart',
 'Illegitimate Barrister',
 'Jakob mark',
 'Kaare',
 'Kasper Holl',
 'Khawabkitabeer',
 'KnudW',
 'Larsbrp',
 'Lindberg',
 'Loveless',
 'Mathiaskh',
 'Mbini~dawiki',
 'Medic',
 'Moeng',
 'Morten Haagensen',
 'Necessary Evil',
 'Nico',
 'Nikolaj',
 'PHE77',
 'Palnatoke',
 'Pandikas',
 'Patchfinder',
 'Pelle S.H.',
 'PerV',
 'Peregrine981',
 'Rmir2',
 'Rune',
 'Sabbe',
 'Sangild',
 'SimmeD',
 'SorenRK',
 'Steenth',
 'TherasTaneel',
 'Trade',
 'Trip Tucker',
 'Villy Fink Isaksen',
 'VseA',
 'Wagner Texas Ranger',
 'Weblars',
 'Zelrin',
 'Zheng Hoi']

### Clustering

In [221]:
adj_matrix = nx.to_numpy_array(G, weight='n_pages')
sc = SpectralClustering(10, affinity='precomputed', n_init=100)
sc.fit(adj_matrix)
print('spectral clustering')
print(sc.labels_)

spectral clustering
[1 1 1 ... 1 1 1]


0

In [236]:
cluster_dict = {node: {'label': label} 
                for node, label in list(zip(list(G.nodes), sc.labels_))}
nx.set_node_attributes(G, cluster_dict)

### Testing clusters for modularity

In [237]:
G.nodes(data=True)

NodeDataView({'AEJ': {'label': 1}, 'Olaf Eller': {'label': 1}, 'Pixi Uno': {'label': 1}, 'Sarrus': {'label': 1}, 'Biscuit-in-Chief': {'label': 1}, 'Christian Giersing': {'label': 1}, 'Crudiant': {'label': 1}, 'Fodbold kongen': {'label': 7}, 'Glenn': {'label': 1}, 'GunnerPoulsen': {'label': 1}, 'Hashar': {'label': 1}, 'HenrikMidtiby': {'label': 1}, 'Hjart': {'label': 1}, 'Ilario': {'label': 1}, 'Inc': {'label': 1}, 'KnudW': {'label': 1}, 'MacApps': {'label': 1}, 'Nielssonnich': {'label': 1}, 'Palnatoke': {'label': 1}, 'Peo': {'label': 1}, 'Pred': {'label': 1}, 'Pugilist': {'label': 1}, 'Rodejong': {'label': 1}, 'Savfisk': {'label': 1}, 'Sten': {'label': 1}, 'Strawa': {'label': 1}, 'Amjaabc': {'label': 1}, 'AndersViborg': {'label': 1}, 'Anne-Sophie Ofrim': {'label': 1}, 'CeciTGreg': {'label': 1}, 'Dipsacus fullonum': {'label': 1}, 'Laanders': {'label': 1}, 'Siksebuffen': {'label': 1}, 'TherasTaneel': {'label': 1}, 'Tjalland': {'label': 1}, 'Tjernobyl': {'label': 1}, 'Toxophilus': {'label

In [242]:
node1 = list(G.nodes)[0]
node2 = list(G.nodes)[3]
nx.get_edge_attributes(G, 'n_pages')[(node1,node2)]

88

In [238]:
len(page_contribs)

426

### Testing text similarities within and between clusters

In [153]:
#user_articles_dict = {user: set(articles) for user, articles 
#                      in list(zip(users,user_articles))}
site = pywikibot.Site('da', 'wikipedia')
pages = pagegenerators.PagesFromTitlesGenerator(page_contribs.keys(), site)
def get_page_plain_text(page_title):
    site = pywikibot.Site('da', 'wikipedia')
    page = pywikibot.Page(site, page_title)
    return page.extract()
    
page_title = list(page_contribs.keys())[0]
wikicode = get_page_plain_text(page_title)

In [137]:
wikicode.filter_text()

['Olaf Eller',
 ' (født 13. juni 1960), er en dansk ',
 'ishockey',
 'træner, ekspertkommentator og forhenværende spiller på det danske ',
 'Danmarks ishockeylandshold',
 'ishockeylandshold',
 '.\n\n',
 ' Aktiv karriere ',
 '\nSom aktiv repræsenterede han i mange år ',
 'Rødovre Skøjte & Ishockey Klub',
 'Rødovre',
 ' 1976-90,  inden den aktive karriere blev rundet af med tre sæsoner , som spillende træner  i ',
 'Rungsted Cobras',
 'Rungsted',
 '. Det var således  i Rungsted, at Ellers trænerkarriere blev indledt, da han i sæsonen 1990-91 var kombineret spiller og træner i den nordsjællandske klub. I denne rolle fortsatte han til 1993\n\nSom spiller har Eller vundet det danske mesterskab 5 gange, alle med Rødovre, i 1978, 1983, 1985, 1986 og 1990. Eller har desuden repræsenteret Danmark ved 104 landskampe.\n\n',
 ' Træner ',
 '\nEfter at have indledt trænerkarrieren mens han stadig var aktiv i Rungsted har Eller i årene derefter trænet mange klubber i den danske ',
 'Superisligaen',
 

In [154]:
get_page_plain_text('Saudi-Arabien')

'Saudi-Arabien (arabisk: السعودية as-Saʿūdīyah), officielt Kongeriget Saudi-Arabien (arabisk: المملكة العربية السعودية Al-Mamlakah al-Arabiyah as-Sa\'ūdiyah), er et land i Mellemøsten og det største land på den Arabiske Halvø.\nLandet grænser mod Jordan i nordvest, Irak i nord, Kuwait i nordøst, Qatar, Bahrain og De Forenede Arabiske Emirater i øst, Oman i sydøst og Yemen i syd. Landet ligger ud til den Persiske Bugt i nordøst og det Røde Hav mod vest og er det eneste land, der har kyster mod begge disse have. Saudi-Arabien har anslået 30,8 millioner indbyggere og et areal på cirka 2.150.000 km².\nKongeriget Saudi-Arabien bliver nogle gange kaldt "De to hellige moskeers land" med tanke på Mekka og Medina, som er muslimernes to helligste byer. Saudi-Arabien blev grundlagt af Abdul Aziz bin Saud, hvis offensiv i 1902 generobrede Al-Saud-familiens forfædres hjemby Riyadh og kulminerede i 1932 med udråbelsen og anerkendelsen af Kongeriget Saudi-Arabien.\nSaudi-Arabien er det land i verden,

In [162]:
stop_words = set(stopwords.words('danish'))

def isabc(string):
    return string.lower() in 'abcdefghijklmnopqrstuvwxyzøæå'

def tokenize_only_abc(text, exclude_stop=True, collocs=[]):
    text = text.lower()
    text = text.replace('\n', ' ')
    text = ''.join([char for char in text if (isabc(char) or char == ' ')])
    tokenizer = MWETokenizer(collocs)
    tokens = tokenizer.tokenize(text.split())
    if exclude_stop:
        return [token for token in tokens if not token in stop_words]
    return tokens

text = get_page_plain_text('Saudi-Arabien')
tokenize_only_abc(text)

['saudiarabien',
 'arabisk',
 'assadyah',
 'officielt',
 'kongeriget',
 'saudiarabien',
 'arabisk',
 'almamlakah',
 'alarabiyah',
 'assadiyah',
 'land',
 'mellemøsten',
 'største',
 'land',
 'arabiske',
 'halvø',
 'landet',
 'grænser',
 'jordan',
 'nordvest',
 'irak',
 'nord',
 'kuwait',
 'nordøst',
 'qatar',
 'bahrain',
 'forenede',
 'arabiske',
 'emirater',
 'øst',
 'oman',
 'sydøst',
 'yemen',
 'syd',
 'landet',
 'ligger',
 'persiske',
 'bugt',
 'nordøst',
 'røde',
 'hav',
 'vest',
 'eneste',
 'land',
 'kyster',
 'begge',
 'saudiarabien',
 'anslået',
 'millioner',
 'indbyggere',
 'areal',
 'cirka',
 'km',
 'kongeriget',
 'saudiarabien',
 'gange',
 'kaldt',
 'to',
 'hellige',
 'moskeers',
 'land',
 'tanke',
 'mekka',
 'medina',
 'muslimernes',
 'to',
 'helligste',
 'byer',
 'saudiarabien',
 'grundlagt',
 'abdul',
 'aziz',
 'bin',
 'saud',
 'offensiv',
 'generobrede',
 'alsaudfamiliens',
 'forfædres',
 'hjemby',
 'riyadh',
 'kulminerede',
 'udråbelsen',
 'anerkendelsen',
 'kongeriget'

In [208]:
def pairwise_cosine(texts):
    vectorizer = CountVectorizer()
    bow_matrix = vectorizer.fit_transform(texts)
    return cosine_similarity(bow_matrix)
# Example list of texts
page_titles = list(page_contribs.keys())[:3]
page_texts = [get_page_plain_text(page_title) for page_title in page_titles]
texts = [' '.join(tokenize_only_abc(text)) for text in page_texts]
print(page_titles)
print(pairwise_cosine(texts))

['Olaf Eller', 'Potentiel energi', 'Josephine Park']
[[1.         0.         0.09877296]
 [0.         1.         0.01495873]
 [0.09877296 0.01495873 1.        ]]


In [217]:
page_titles = list(page_contribs.keys())

def tokenized_text_from_page_title(page_title):
    page_text = get_page_plain_text(page_title)
    tokenized_text = ' '.join(tokenize_only_abc(page_text))
    return tokenized_text

def mean_cosine_distance(page_titles):
    n_pages = len(page_titles)
    tokenized_texts = Parallel(n_jobs=-2)(delayed(tokenized_text_from_page_title)(page_title) for page_title in tqdm(page_titles))
    cosine_distances = pairwise_cosine(tokenized_texts)
    mean_cosine = (np.sum(cosine_distances) - n_pages) / (2 * n_pages)
    return mean_cosine
mean_cosine_distance(page_titles)

100%|██████████| 20/20 [00:00<00:00, 59.90it/s]


calculating


0.2707151766684177

0.037333333333333336

### Finding out what the clusters mean by keywords